
# NeuralForecast ロト特徴量ワークベンチ

このノートブックでは、次のような操作を **Jupyter 上だけで** 行えるようにします。

1. プロジェクトルートのパス設定 & `db_config` の import 確認  
2. PostgreSQL への接続テスト  
3. `nf_loto%` 系テーブル一覧の確認  
4. `nf_loto_final`・特徴量テーブル（`nf_loto_hist_exog` など）のプレビュー  
5. `loto_feature_builder.py` の実行（特徴量生成バッチをノートブックから叩く）  
6. NeuralForecast に渡すためのデータ形（`df`, `futr_df`, `static_df`）のイメージ確認  

各セルには簡単な解説コメントを付けてあります。



## 1. プロジェクトルートのパス設定

環境によっては Jupyter を起動したカレントディレクトリが異なるため、  
`db_config.py` を確実に import できるように `sys.path` を明示的に通します。

- 通常は `C:\nf\nf_loto_feature_project` をプロジェクトルートとして想定しています。
- このセルを実行して、`project_root` が正しいディレクトリを指していることを確認してください。


In [ ]:

import os, sys, pathlib

# --- ここを自分の環境に合わせて変更しても良い ---
# None の場合は「現在の作業ディレクトリ」から自動推定
EXPLICIT_PROJECT_ROOT = None  # 例: r"C:\nf\nf_loto_feature_project"

if EXPLICIT_PROJECT_ROOT is not None:
    project_root = pathlib.Path(EXPLICIT_PROJECT_ROOT).resolve()
else:
    # 現在の作業ディレクトリから notebooks/ の中かどうかを判定
    cwd = pathlib.Path(os.getcwd()).resolve()
    if cwd.name == "notebooks":
        project_root = cwd.parent
    else:
        project_root = cwd

# sys.path に追加
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print("project_root:", project_root)
print("sys.path[0]:", sys.path[0])



## 2. `db_config` の import と DB 接続テスト

このセルでは:

- `db_config.DB_CONFIG` を import できるか
- `psycopg2.connect(**DB_CONFIG)` で DB に接続できるか

を確認します。  
接続情報そのもの（パスワードなど）は画面に表示しません。


In [ ]:

import psycopg2
from db_config import DB_CONFIG

print("DB_CONFIG のキー:", DB_CONFIG.keys())

try:
    conn = psycopg2.connect(**DB_CONFIG)
    print("PostgreSQL への接続に成功しました。")
    conn.close()
except Exception as e:
    print("PostgreSQL への接続に失敗しました。")
    print("エラー内容:", e)



## 3. `nf_loto%` テーブル一覧の確認

ロト関連テーブル (`nf_loto_final`, `nf_loto_hist_exog` など) が  
どのくらい作成されているかを一覧表示します。


In [ ]:

import pandas as pd
import psycopg2
from db_config import DB_CONFIG

conn = psycopg2.connect(**DB_CONFIG)
with conn:
    df_tables = pd.read_sql("""
        SELECT tablename
        FROM pg_catalog.pg_tables
        WHERE schemaname NOT IN ('pg_catalog', 'information_schema')
          AND tablename LIKE 'nf_loto%'
        ORDER BY tablename
    """, conn)
conn.close()

df_tables



## 4. `nf_loto_final` の中身を確認

ロトの元テーブル `nf_loto_final` から、先頭数十行だけ確認します。  
（テーブル名が存在しない場合はエラーになります）


In [ ]:

import pandas as pd
import psycopg2
from db_config import DB_CONFIG

target = "nf_loto_final"  # 必要に応じて変更

conn = psycopg2.connect(**DB_CONFIG)
with conn:
    df_final = pd.read_sql(f"SELECT * FROM {target} ORDER BY ds, unique_id LIMIT 50", conn)
conn.close()

df_final.head()



## 5. 特徴量テーブルの確認 (`nf_loto_hist_exog` など)

特徴量生成バッチ (`loto_feature_builder.py`) を実行済みであれば、  
`nf_loto_hist_exog` / `nf_loto_futr_exog` / `nf_loto_stat_exog` / `nf_loto_y_features` が存在する想定です。  
ここでは存在チェックをしつつ、あれば中身をプレビューします。


In [ ]:

import pandas as pd
import psycopg2
from db_config import DB_CONFIG

conn = psycopg2.connect(**DB_CONFIG)

def preview_table(table_name: str, limit: int = 20):
    print(f"--- {table_name} ---")
    with conn:
        df = pd.read_sql(
            f"""SELECT * FROM {table_name} ORDER BY 1 LIMIT %s""", conn,
            params=(int(limit),)
        )
    display(df.head())

# まず nf_loto% テーブル一覧を取得
with conn:
    df_tables = pd.read_sql("""
        SELECT tablename
        FROM pg_catalog.pg_tables
        WHERE schemaname NOT IN ('pg_catalog', 'information_schema')
          AND tablename LIKE 'nf_loto%'
        ORDER BY tablename
    """, conn)

tables = set(df_tables["tablename"].tolist())

for t in ["nf_loto_hist_exog", "nf_loto_futr_exog", "nf_loto_stat_exog", "nf_loto_y_features"]:
    if t in tables:
        preview_table(t)
    else:
        print(f"{t} はまだ存在しません。")
        
conn.close()



## 6. ノートブックから特徴量生成バッチを実行する

`loto_feature_builder.py` の `main()` を直接呼び出すことで、  
このノートブック上から特徴量テーブル生成バッチを実行できます。

- 元データテーブル `nf_loto_final` が空の場合は、その旨のメッセージを出して終了します。
- 実行ログは標準出力に表示されます。


In [ ]:

from loto_feature_builder import main as run_loto_feature_builder

# 実行には時間がかかることがあります。
run_loto_feature_builder()



## 7. NeuralForecast への入力形のイメージ

NeuralForecast に渡す典型的なデータの形は次の通りです。

- 学習用 df（過去データ）
  - カラム: `['unique_id', 'ds', 'y', hist_..., futr_(過去分), ...]`
- 未来用 futr_df（予測期間の未来既知外生）
  - カラム: `['unique_id', 'ds', futr_...]`
- 静的外生 static_df
  - カラム: `['unique_id', stat_...]`

ここでは、`nf_loto_final` と各特徴量テーブルを join して、  
NeuralForecast に食わせられそうな DataFrame を組み立てる例を示します。


In [ ]:

import pandas as pd
import psycopg2
from db_config import DB_CONFIG

conn = psycopg2.connect(**DB_CONFIG)

# ベース (y) テーブル
with conn:
    df_y = pd.read_sql("""
        SELECT loto, unique_id, ds, y
        FROM nf_loto_final
    """, conn)

# 履歴特徴
try:
    with conn:
        df_hist = pd.read_sql("""
            SELECT *
            FROM nf_loto_hist_exog
        """, conn)
except Exception as e:
    print("nf_loto_hist_exog が無いか読み込みに失敗しました:", e)
    df_hist = None

# 静的特徴
try:
    with conn:
        df_stat = pd.read_sql("""
            SELECT *
            FROM nf_loto_stat_exog
        """, conn)
except Exception as e:
    print("nf_loto_stat_exog が無いか読み込みに失敗しました:", e)
    df_stat = None

conn.close()

# join の例: y + hist
if df_hist is not None:
    df_train = pd.merge(df_y, df_hist, on=["loto", "unique_id", "ds"], how="left")
else:
    df_train = df_y.copy()

df_train.head()



上記 `df_train` から、NeuralForecast 用には例えば次のように列を分けます：

```python
futr_exog_list = [c for c in df_train.columns if c.startswith("futr_")]
hist_exog_list = [c for c in df_train.columns if c.startswith("hist_")]

# 学習用 df 例:
df_for_nf = df_train[["unique_id", "ds", "y"] + futr_exog_list + hist_exog_list]

# 静的外生:
stat_exog_list = [c for c in df_stat.columns if c.startswith("stat_")]
static_df = df_stat[["unique_id"] + stat_exog_list]
```
